In [ ]:
import glob
import random
import os
import natsort
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.transforms as transforms

In [ ]:
import argparse # parser
from os.path import basename #path
import numpy as np #numpy
import torch #torch
import cv2 # cv
import torchvision #torch vision
import colorsys
import random
from random import randint
import glob
import time
import natsort # sort file list by number
from torch.utils.data import DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from scipy import ndimage
import shutil

# DataLoader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_ = None):
        self.transform = transforms.Compose(transforms_)
        self.files_A = sorted(glob.glob(os.path.join(root ) + '*.*'))        
        self.files_A = natsort.natsorted(self.files_A)        
    def __getitem__(self, index):
        
        item_A = self.transform(Image.open(self.files_A[index]))        
        return {'A': item_A}
        
    def __len__(self):
        return len(self.files_A)

In [ ]:
print("torch vision version:" +   torchvision.__version__)
print("torch version:" + torch.__version__)
print("torch cuda version:" + torch.version.cuda)

torch vision version:0.14.0+cu116
torch version:1.13.0+cu116
torch cuda version:11.6


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--batchSize', type=int, default=1, help='size of the batches')
parser.add_argument('--root', type = str, default = './', help = 'root dir')
parser.add_argument('--data_path', type = str, default = './images/' , help = 'dataset dir')
parser.add_argument('--save_path', type = str, default = './output/' , help = 'result path')

_StoreAction(option_strings=['--save_path'], dest='save_path', nargs=None, const=None, default='./output/', type=<class 'str'>, choices=None, help='result path', metavar=None)

In [ ]:
parser.add_argument('--cuda', action = 'store_false', help = 'use GPU computation')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--input_nc', type=int, default=3, help='number of channels of input data')

_StoreAction(option_strings=['--input_nc'], dest='input_nc', nargs=None, const=None, default=3, type=<class 'int'>, choices=None, help='number of channels of input data', metavar=None)

In [ ]:
parser.add_argument('-f')
opt = parser.parse_args()

 # 모델 구조 파악

In [ ]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
# 모델에서 분류할 수 있는 class의 name을 list로 만듦.
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
#######################################
# Networks Definition
# TorchVision에 존재하는 maskrcnn을 사용하고자 함.
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
#if torch.cuda.is_available():
#  print("WARNING: You have a CUDA device, so you should probably run with --cuda")
#  model.cuda()
#######################################


#####################################################
# Set model's test mode
model.eval()

In [ ]:
def init(data_path):
    #Dataset Loader
    transforms_ = [ transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
    dataloader = DataLoader(ImageDataset(data_path, transforms_=transforms_), 
                            batch_size=opt.batchSize, shuffle=False, num_workers=opt.n_cpu)
    return dataloader

In [ ]:
def human_remove(image,mask,labels,file_name,save_path):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)    
    image = cv2.cvtColor(image,cv2.COLOR_RGB2RGBA)
    
    image[:,:,3] = 255.0
    channel = image.shape[2]
    area = 0    
    _,_,w,h = mask.shape
    for n in range(mask.shape[0]):
            if labels[n] == 1:   #1이 human임 !!             
                mask[n] = np.where(mask[n] >0.5, 255,0)
            else :
                continue        
    for n in range(mask.shape[0]):
            if labels[n] == 1:
                for c in range(channel):                
                    image[:,:,c] = np.where(mask[n] == 255, 0, image[:,:,c])
            else :
                continue                
    #image save
    cv2.imwrite(save_path +'seg_' + file_name ,image)

In [ ]:
# mask를 적용해 출력하고자 할때 사용
def apply_mask(image,mask,labels,boxes,file_name,save_path):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)    
    # image = cv2.cvtColor(image,cv2.COLOR_RGB2RGBA)
    
    alpha = 1 
    beta = 0.6 # transparency for the segmentation map
    gamma = 0 # scalar added to each sum
    COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))    
    channel = image.shape[2]
    area = 0    
    _,_,w,h = mask.shape    
    segmentation_map = np.zeros((w,h,3),np.uint8)
    
    for n in range(mask.shape[0]): 
        if labels[n] == 0:
            continue
        else:
            color = COLORS[random.randrange(0,len(COLORS))]
            segmentation_map[:,:,0] = np.where(mask[n] > 0.5, COLORS[labels[n]][0], 0)
            segmentation_map[:,:,1] = np.where(mask[n] > 0.5, COLORS[labels[n]][1], 0)
            segmentation_map[:,:,2] = np.where(mask[n] > 0.5, COLORS[labels[n]][2], 0)            
            image = cv2.addWeighted(image,alpha,segmentation_map,beta,gamma,dtype = cv2.CV_8U)
        # draw the bounding boxes around the objects        
        cv2.rectangle(image, boxes[n][0], boxes[n][1],color = color ,thickness = 2)
        
        print(class_names[labels[n]])
        # put the label text above the objects
        cv2.putText(image , class_names[labels[n]], (boxes[n][0][0], boxes[n][0][1]-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                    thickness=2, lineType=cv2.LINE_AA)
    #image save
    cv2.imwrite(save_path +'seg_2' + file_name ,image) 

In [ ]:
# mask의 색을 random하게 배정하기 위한 함수 
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

In [ ]:
#텐서를 맞는 형태로 변형
def tensor2im(tensor):
    # if opt.cuda == True:
        # tensor = 127.5*(tensor[0].data.cpu().float().numpy() +1.0)
    # else:
    tensor = 127.5 *(tensor[0].data.float().numpy() +1.0)
    img = tensor.astype(np.uint8) #tensor의 type을 unit8로 변경
    img = np.transpose(tensor,(1,2,0))    
    return img

In [ ]:
def mask_rcnn(file_list,dataloader,save_path):
    for i,batch in enumerate(dataloader):        
    
        img = batch['A']      
        print(file_list[i])
        
        #use cuda
        #result = model(img.to('cuda'))
        #use cpu
        result = model(img) #이미지를 maskRCNN에 넣은 결과를 result에 넣게 됨.
        
        image = tensor2im(img) #apply_mask, human_remove 등을 하기 위한 tensor 생성

        scores = list(result[0]['scores'].detach().numpy())
        thresholded_preds_inidices = [scores.index(i) for i in scores if i > 0.965] # score값이 0.965 이상에 해당하는 것들만 넣고자 함.
        thresholded_preds_count = len(thresholded_preds_inidices)   #0.965 이상인 것들의 수를 따로 저장     
        mask = result[0]['masks']
        mask = mask[:thresholded_preds_count] # 각각의 것에 대한 mask 진행
        labels = result[0]['labels']        # label을 붙임
        boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))]  for i in result[0]['boxes']]
        boxes = boxes[:thresholded_preds_count]
        
        mask = mask.data.float().numpy()        
        #사람을 지우고 싶을때
        # human_remove(image,mask,labels,file_list[i],save_path)
        # 모든 객체에 segmetation과 박스를 출력하고 싶을때
        apply_mask(image,mask,labels,boxes,file_list[i],save_path)

In [ ]:
def apply_mask(image,mask,labels,boxes,file_name,save_path):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)    
    # image = cv2.cvtColor(image,cv2.COLOR_RGB2RGBA)
    
    alpha = 1 
    beta = 0.6 # transparency for the segmentation map -> 투명도 조절
    gamma = 0 # scalar added to each sum
    COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))    # 랜덤색상 선정
    channel = image.shape[2]
    area = 0    
    _,_,w,h = mask.shape    
    segmentation_map = np.zeros((w,h,3),np.uint8)
    
    for n in range(mask.shape[0]): 
        if labels[n] == 0:
            continue
        else:
            color = COLORS[random.randrange(0,len(COLORS))]
            segmentation_map[:,:,0] = np.where(mask[n] > 0.5, COLORS[labels[n]][0], 0)
            segmentation_map[:,:,1] = np.where(mask[n] > 0.5, COLORS[labels[n]][1], 0)
            segmentation_map[:,:,2] = np.where(mask[n] > 0.5, COLORS[labels[n]][2], 0)            
            image = cv2.addWeighted(image,alpha,segmentation_map,beta,gamma,dtype = cv2.CV_8U)
        # draw the bounding boxes around the objects        
        cv2.rectangle(image, boxes[n][0], boxes[n][1],color = color ,thickness = 2)
        
        print(class_names[labels[n]])
        # put the label text above the objects
        cv2.putText(image , class_names[labels[n]], (boxes[n][0][0], boxes[n][0][1]-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color, 
                    thickness=2, lineType=cv2.LINE_AA)
    #image save
    cv2.imwrite(save_path +'seg_2' + file_name ,image)

In [ ]:
def main():
    if not os.path.exists(opt.save_path):
            os.makedirs(opt.save_path)
    if not os.path.exists(opt.data_path):
        os.makedirs(opt.data_path)
    file_list = os.listdir(opt.data_path)
    file_list = natsort.natsorted(file_list)    
    dataloader = init(opt.data_path)
    mask_rcnn(file_list,dataloader,opt.save_path) 

In [ ]:
if __name__ == "__main__":    
    main()

1111.jpg
person
car
person
person
